In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
test_data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

### date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

In [ ]:
train

In [ ]:
test_data

In [ ]:
train['date'] = pd.to_datetime(train['date'],format = '%d.%m.%Y')
train

# Pivot Table - index =? , value = ?, columns =?

In [ ]:
dataset = train.pivot_table(index = ['shop_id', 'item_id'], values = ['item_cnt_day'],
                           columns = ['date_block_num'], fill_value = 0, aggfunc = 'sum')

In [ ]:
dataset

In [ ]:
dataset.reset_index(inplace = True)
dataset.head(20)

In [ ]:
test_data.head(10)

In [ ]:
dataset = pd.merge(test_data, dataset, on = ['item_id', 'shop_id'],
                  how = 'left')

In [ ]:
dataset

In [ ]:
dataset.fillna(0, inplace = True)
dataset.head()

In [ ]:
dataset.drop(['shop_id', 'item_id', 'ID'], inplace = True, axis = 1)
dataset

In [ ]:
# 0~ 32 까지 학습셋
X_train = np.expand_dims(dataset.values[:, :-1], axis =2)

# 33 (34번째 컬럼) --> 라벨값, 정답값
y_train = dataset.values[:, -1:]

# 1~ 33 까지가 평가셋 (테스트셋) --> 예측하면 34 (35번째 컬럼) 에 대한 예측값이 나옴
X_test = np.expand_dims(dataset.values[:, 1:], axis =2)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
model = Sequential()

model.add(LSTM(units = 64, input_shape = (33,1)))
model.add(Dropout(0.4))
model.add(Dense(1))

In [ ]:
model.compile(metrics = ['mean_squared_error'], loss = 'mse', 
              optimizer = 'adam')

In [ ]:
model.fit(X_train, y_train, batch_size = 1024, epochs = 3)

In [ ]:
submission = model.predict(X_test)

In [ ]:
submission

In [ ]:
submission.shape

In [ ]:
import seaborn as sns

sns.boxplot(y_train)

In [ ]:
y2 = y_train[y_train < 100]

y3 = pd.DataFrame(columns =['y'])

y3['y'] = y2

sns.distplot(y3)

In [ ]:
y2.mean()

In [ ]:
submission = submission.clip(0,20)

In [ ]:
submission = pd.DataFrame({'ID' : test_data['ID'],
                          'item_cnt_month' : submission.ravel()})

In [ ]:
submission

In [ ]:
submission.to_csv('baseline.csv', index = False)